# Ubiquant Market Visualize Importance & Predict

In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm
import category_encoders as ce
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import plotly.express as px
import optuna


# Data preparation

In [ ]:
adopt0=[ 'f_165', 'f_197', 'f_231', 'f_225', 'f_249', 
       'f_250', 'f_73', 'f_175', 'f_61', 'f_170', 'f_145', 'f_3', 'f_7', 
       'f_153', 'f_67', 'f_182', 'f_42', 'f_233', 'f_292', 'f_25', 'f_299', 
       'f_179', 'f_63', 'f_137', 'f_127', 'f_275', 'f_184', 'f_240', 'f_110', 
       'f_19', 'f_200', 'f_122', 'f_265', 'f_60', 'f_264', 'f_128', 'f_202', 
       'f_270', 'f_251', 'f_149', 'f_16', 'f_22', 'f_26', 'f_125', 'f_64', ]

In [ ]:
data0a = pd.read_csv('../input/ubiquant-market-train-data-splitting/train0a.csv')
data0a2=data0a[adopt0+['target']]
del data0a
data0b = pd.read_csv('../input/ubiquant-market-train-data-splitting/train0b.csv')
data0b2=data0b[adopt0+['target']]
del data0b
data0c = pd.read_csv('../input/ubiquant-market-train-data-splitting/train0c.csv')
data0c2=data0c[adopt0+['target']]
del data0c
data0d = pd.read_csv('../input/ubiquant-market-train-data-splitting/train0d.csv')
data0d2=data0d[adopt0+['target']]
del data0d

In [ ]:
data0 = pd.concat([data0a2,data0b2,data0c2,data0d2],axis=0)
print(data0.shape)

In [ ]:
df=data0.copy()
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
    if df[c].dtype=='object': 
        df[c] = df[c].fillna('N')
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [ ]:
data1=df
data1[0:2].T

In [ ]:
data1.info()

# Target setting

In [ ]:
target=['target']
dataY=data1['target']
dataX=data1.drop('target',axis=1)

In [ ]:
trainX=np.array(dataX)
trainY=np.array(dataY)
print(trainX.shape)
print(trainY.shape)

In [ ]:
df_columns = list(dataX.columns)
print(df_columns)

In [ ]:
train_df=pd.DataFrame(trainX)
#test_df=pd.DataFrame(test)
train_df.columns=df_columns
#test_df.columns=df_columns
print(train_df.shape)

In [ ]:
def create_numeric_feature(input_df):
    use_columns = df_columns 
    return input_df[use_columns].copy()

In [ ]:
from contextlib import contextmanager
from time import time

class Timer:
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' '):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [ ]:
from tqdm import tqdm

def to_feature(input_df):

    processors = [
        create_numeric_feature,
    ]
    
    out_df = pd.DataFrame()
    
    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='create' + func.__name__ + ' '):
            _df = func(input_df)

        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
        
    return out_df

In [ ]:
train_feat_df = to_feature(train_df)
#test_feat_df = to_feature(test_df)
print(train_feat_df.shape)

# Model

In [ ]:
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error

def fit_lgbm(X, y, cv, 
             params: dict=None, 
             verbose: int=50):

    if params is None:
        params = {}

    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X[idx_train], y[idx_train]
        x_valid, y_valid = X[idx_valid], y[idx_valid]

        clf = lgbm.LGBMRegressor(**params)
        
        with Timer(prefix='fit fold={} '.format(i)):
            clf.fit(x_train, y_train, 
                    eval_set=[(x_valid, y_valid)],  
                    early_stopping_rounds=100,
                    verbose=verbose)

        pred_i = clf.predict(x_valid)
        oof_pred[idx_valid] = pred_i
        models.append(clf)
        print(f'Fold {i} RMSLE: {mean_squared_error(y_valid, pred_i) ** .5:.4f}')
        print()

    score = mean_squared_error(y, oof_pred) ** .5
    print('-' * 50)
    print('FINISHED | Whole RMSLE: {:.4f}'.format(score))
    return oof_pred, models

In [ ]:
params = {
    'objective': 'rmse', 
    'learning_rate': .1,
    'reg_lambda': 1.,
    'reg_alpha': .1,
    'max_depth': 5, 
    'n_estimators': 10000, 
    'colsample_bytree': .5, 
    'min_child_samples': 10,
    'subsample_freq': 3,
    'subsample': .9,
    'importance_type': 'gain', 
    'random_state': 71,
    'num_leaves': 62
}

In [ ]:
y = trainY
ydf=pd.DataFrame(y)

In [ ]:
from sklearn.model_selection import KFold

for i in range(1):
    fold = KFold(n_splits=5, shuffle=True, random_state=71)
    ydfi=ydf.iloc[:,i]
    y=np.array(ydfi)
    cv = list(fold.split(train_feat_df, y))
    oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
    
    fig,ax = plt.subplots(figsize=(6,6))
    ax.set_title(target[i],fontsize=20)
    ax.set_ylabel('Predicted target',fontsize=12)
    ax.set_xlabel('Actual target',fontsize=12)
    ax.scatter(oof,y)


# Visualize Importance

In [ ]:
def visualize_importance(models, feat_train_df):

    feature_importance_df = pd.DataFrame()
    for i, model in enumerate(models):
        _df = pd.DataFrame()
        _df['feature_importance'] = model.feature_importances_
        _df['column'] = feat_train_df.columns
        _df['fold'] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, _df], 
                                          axis=0, ignore_index=True)

    order = feature_importance_df.groupby('column')\
        .sum()[['feature_importance']]\
        .sort_values('feature_importance', ascending=False).index[0:100]
    
    adopt=order.tolist()
    print('The importance 100')
    print(adopt)

    fig, ax = plt.subplots(figsize=(8, max(6, len(order) * .25)))
    sns.boxenplot(data=feature_importance_df, 
                  x='feature_importance', 
                  y='column', 
                  order=order, 
                  ax=ax, 
                  palette='viridis', 
                  orient='h')
    
    ax.tick_params(axis='x', rotation=0)
    #ax.set_title('Importance')
    ax.grid()
    fig.tight_layout()
    
    return fig,ax

#fig, ax = visualize_importance(models, train_feat_df)

In [ ]:
fold = KFold(n_splits=5, shuffle=True, random_state=71)
ydfi=ydf.iloc[:,0]
y=np.array(ydfi)
cv = list(fold.split(train_feat_df, y))
oof, models = fit_lgbm(train_feat_df.values, y, cv, params=params, verbose=500)
fig, ax = visualize_importance(models, train_feat_df)
ax.set_title('Target Imortance',fontsize=20)

In [ ]:
fig,ax = plt.subplots(figsize=(16,6))
ax.set_title('f_165 vs target',fontsize=20)
ax.set_xlabel('f_165',fontsize=12)
ax.set_ylabel('target',fontsize=12)
ax.scatter(data0['f_165'],data0['target'])
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(16,6))
ax.set_title('f_249 vs target',fontsize=20)
ax.set_xlabel('f_249',fontsize=12)
ax.set_ylabel('target',fontsize=12)
ax.scatter(data0['f_249'],data0['target'])
plt.show()

In [ ]:
test0=pd.read_csv('../input/ubiquant-market-prediction/example_test.csv')
submit=pd.read_csv('../input/ubiquant-market-prediction/example_sample_submission.csv')

In [ ]:
test0=test0[adopt0]
print(test0.shape)

In [ ]:
df=test0.copy()
from sklearn.preprocessing import LabelEncoder
for c in df.columns:
    if df[c].dtype=='object': 
        df[c] = df[c].fillna('N')
        lbl = LabelEncoder()
        lbl.fit(list(df[c].values))
        df[c] = lbl.transform(df[c].values)

In [ ]:
test1=df
print(test1.columns.tolist())

In [ ]:
models

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

for (test_df, sample_prediction_df) in iter_test:
    
    test_df['target'] = 0
    
    for model in models:
        test_df['target'] += model.predict(test_df[adopt0])
    
    test_df['target'] /= len(models)
    env.predict(test_df[['row_id','target']])